In [1]:
import pandas
import datetime
import numpy
import scipy.optimize
import seaborn as sns
import matplotlib.pyplot as plt
import os
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
%matplotlib inline

In [41]:
# Read the ExB drift and sc potential data
exbFile = "/Users/bharat/Documents/code/vap-analys/" +\
            "data/SAIS_SAPS_withdata/selected_SAPS/" +\
            "A_130622/SAPS_data_ExB_iono_A_2013062205000700.txt"
scPotFile = "/Users/bharat/Documents/code/vap-analys/" +\
            "data/SAIS_SAPS_withdata/selected_SAPS/" +\
            "A_130622/SAPS_data_SCPOT_A_2013062205000700.txt"
exbColNames = [ "dec_hour", "radial_vel", "azim_vel" ]
scPotColNames = [ "dec_hour", "sc_potential" ]
exbRbspDF = pandas.read_csv(exbFile, delim_whitespace=True,\
                            header=None, names=exbColNames)
scPotRbspDF = pandas.read_csv(scPotFile, delim_whitespace=True,\
                            header=None, names=scPotColNames)
# Add a datetime field from the dec_hour field
scPotRbspDF["hour"] = [ int(x) for x in scPotRbspDF["dec_hour"] ]
scPotRbspDF["minute"] = [ int( round( ( x - int(x) ) * 60. ) ) for x in scPotRbspDF["dec_hour"] ]
scPotRbspDF["dec_minute"] = [ ( x - int(x) ) * 60. for x in scPotRbspDF["dec_hour"] ]

scPotRbspDF["dec_minute2"] = [ ( x )  for x in scPotRbspDF["dec_minute"] ]

scPotRbspDF["date"] = [ datetime.datetime(2013,6,22,int(x)) for x in scPotRbspDF["dec_hour"] ]
scPotRbspDF.head(50)

,dec_hour,sc_potential,hour,minute,dec_minute,dec_minute2,date
0,5.00097,-6.89928,5,0,0.0582,0.0582,2013-06-22 05:00:00
1,5.00405,-6.85098,5,0,0.2430,0.2430,2013-06-22 05:00:00
2,5.00713,-6.82217,5,0,0.4278,0.4278,2013-06-22 05:00:00
3,5.01021,-6.97612,5,1,0.6126,0.6126,2013-06-22 05:00:00
4,5.01329,-7.01915,5,1,0.7974,0.7974,2013-06-22 05:00:00
5,5.01637,-6.93820,5,1,0.9822,0.9822,2013-06-22 05:00:00
6,5.01945,-6.78012,5,1,1.1670,1.1670,2013-06-22 05:00:00
7,5.02253,-6.65080,5,1,1.3518,1.3518,2013-06-22 05:00:00
8,5.02561,-6.64714,5,2,1.5366,1.5366,2013-06-22 05:00:00
9,5.02869,-6.69376,5,2,1.7214,1.7214,2013-06-22 05:00:00
